In [1]:
import pandas as pd
import streamlit as st
from sklearn.linear_model import LogisticRegression #explicarei mais a frente o motivo de ser regressão logística

In [2]:
data_model_01 = pd.read_csv('dados_pro_target_model.csv') # dados target produtividade
data_model_02 = pd.read_csv('variáveis_explicativas_agro.csv') # variávveis explicativas

In [53]:
data_model_02 = data_model_02[['QV2M','T2M_RANGE']]


In [54]:
dados_model = pd.merge(data_model_02, data_model_01[['Target_Produtividade']], left_index=True, right_index=True, how='inner')

In [55]:
dados_model = pd.DataFrame(dados_model)

## foda

In [57]:

explicativas = dados_model.drop(axis=1,columns='Target_Produtividade')
resposta  = dados_model['Target_Produtividade']



In [59]:
print(dados_model.columns)


Index(['QV2M', 'T2M_RANGE', 'Target_Produtividade'], dtype='object')


In [60]:
from sklearn.model_selection import train_test_split ## parei aqui


x_train, x_test, y_train, y_test = train_test_split(explicativas, resposta, test_size = 0.3, random_state = 0)

In [61]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((244, 2), (105, 2), (244,), (105,))

In [62]:
logreg = LogisticRegression(penalty='l2', 
                            dual=False, 
                            tol=0.0001, 
                            C=1, 
                            fit_intercept=True, 
                            intercept_scaling=1, 
                            class_weight=None, 
                            random_state=None, 
                            solver='liblinear', 
                            max_iter=100, 
                            multi_class='ovr', 
                            verbose=0, 
                            warm_start=False, 
                            n_jobs=-1)
logreg.fit(x_train, y_train)


# Treino
y_pred_logreg_train = logreg.predict(x_train)
y_score_logreg_train = logreg.predict_proba(x_train)

# Teste
y_pred_logreg_test = logreg.predict(x_test)
y_score_logreg_test = logreg.predict_proba(x_test)


ValueError: Unknown label type: continuous. Maybe you are trying to fit a classifier, which expects discrete classes on a regression target with continuous values.

In [65]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error



# Divida os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(explicativas, resposta, test_size=0.3, random_state=42)

# Use um modelo de regressão linear como exemplo
modelo = LinearRegression()

# Treine o modelo
modelo.fit(X_train, y_train)

# Faça previsões
previsoes = modelo.predict(X_test)

# Avalie o desempenho do modelo (substitua pela métrica apropriada para o seu problema)
erro = mean_squared_error(y_test, previsoes)

print(f'Erro: {erro}')



Erro: 61384.96919682279


In [69]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix

# Exemplo de dados (substitua isso pelos seus dados)
X, y = explicativas,resposta
# Divida os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Use um modelo de Regressão Logística
modelo = LogisticRegression()

# Treine o modelo
modelo.fit(X_train, y_train)

# Faça previsões
previsoes = modelo.predict(X_test)

# Avalie o desempenho do modelo
acuracia = accuracy_score(y_test, previsoes)
matriz_confusao = confusion_matrix(y_test, previsoes)

print(f'Acurácia: {acuracia}')
print(f'Matriz de Confusão:\n{matriz_confusao}')


ValueError: Unknown label type: continuous. Maybe you are trying to fit a classifier, which expects discrete classes on a regression target with continuous values.

In [70]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error

# Exemplo de dados (substitua isso pelos seus dados)
X, y = explicativas,resposta

# Divida os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Use um modelo de Regressão de Árvore de Decisão
modelo = DecisionTreeRegressor()

# Treine o modelo
modelo.fit(X_train, y_train)

# Faça previsões
previsoes = modelo.predict(X_test)

# Avalie o desempenho do modelo
erro_mse = mean_squared_error(y_test, previsoes)

print(f'Erro Médio Quadrático (MSE): {erro_mse}')


Erro Médio Quadrático (MSE): 140131.68890380953


In [76]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Exemplo de dados (substitua isso pelos seus dados)
X, y = explicativas,resposta

# Divida os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Use um modelo de regressão linear como exemplo
modelo = LinearRegression()

# Treine o modelo
modelo.fit(X_train, y_train)

# Faça previsões
previsoes = modelo.predict(X_test)

# Avalie o desempenho do modelo (substitua pela métrica apropriada para o seu problema)
erro = mean_squared_error(y_test, previsoes)

print(f'Erro: {erro}')


Erro: 62116.84179849007


In [77]:
# 1) Cálculo da acurácia
from sklearn.metrics import accuracy_score

#Treino
acc_logreg_train = round(accuracy_score(y_pred_logreg_train, y_train) * 100, 2)

#Teste
acc_logreg_test = round(accuracy_score(y_pred_logreg_test, y_test) * 100, 2)

# 2) Cálculo da área sob curva ROC e Gini
from sklearn.metrics import roc_curve, auc

# Treino
fpr_logreg_train, tpr_logreg_train, thresholds = roc_curve(y_train, y_score_logreg_train[:,1])
roc_auc_logreg_train = 100*round(auc(fpr_logreg_train, tpr_logreg_train), 2)
gini_logreg_train = 100*round((2*roc_auc_logreg_train/100 - 1), 2)

# Teste
fpr_logreg_test, tpr_logreg_test, thresholds = roc_curve(y_test, y_score_logreg_test[:,1])
roc_auc_logreg_test = 100*round(auc(fpr_logreg_test, tpr_logreg_test), 2)
gini_logreg_test = 100*round((2*roc_auc_logreg_test/100 - 1), 2)


# 3) Gráfico da curva ROC
import matplotlib.pyplot as plt

plt.style.use('seaborn-darkgrid')
plt.figure(figsize=(12,6))

lw = 2

plt.plot(fpr_logreg_train, tpr_logreg_train, color='blue',lw=lw, label='ROC (Treino = %0.0f)' % roc_auc_logreg_train)
plt.plot(fpr_logreg_test, tpr_logreg_test, color='darkorange',lw=lw, label='ROC (Teste = %0.0f)' % roc_auc_logreg_test)

plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Falso Positivo', fontsize=15)
plt.ylabel('Verdadeiro Positivo', fontsize=15)
plt.legend(loc="lower right")
plt.legend(fontsize=20) 
plt.title('Curva ROC - Modelo Titanic Kaggle', fontsize=20)
plt.show()

print('Acurácia, Gini e Área Curva ROC (Base de Treino): ',acc_logreg_train, gini_logreg_train, roc_auc_logreg_train)
print('Acurácia, Gini e Área Curva ROC (Base de Teste): ',acc_logreg_test, gini_logreg_test, roc_auc_logreg_test)

NameError: name 'y_pred_logreg_train' is not defined

In [71]:
from sklearn import svm

In [72]:
regr = svm.SVR()

In [73]:
regr.fit(X, y)

SVR()

In [75]:
regr.predict(explicativas)

array([2823.76461318, 2822.59473158, 2819.874138  , 2819.75234168,
       2822.2395503 , 2822.9827603 , 2823.25149286, 2823.11844332,
       2822.88410276, 2821.5510595 , 2821.10486068, 2819.81771083,
       2821.82414499, 2822.32015698, 2823.14698026, 2823.21014507,
       2823.12804752, 2823.61921625, 2822.38042002, 2823.03615503,
       2821.08889542, 2820.41967462, 2820.72002368, 2821.3923379 ,
       2818.84898397, 2818.34277382, 2818.54180176, 2822.25429709,
       2820.85817131, 2820.1395973 , 2821.97793703, 2818.18612563,
       2823.40523027, 2821.99204116, 2820.4156398 , 2818.6261721 ,
       2821.0523944 , 2823.24748407, 2821.79392503, 2820.47838198,
       2822.87365341, 2821.37679674, 2821.36561826, 2821.31237215,
       2818.67238676, 2821.64037046, 2820.30124686, 2820.53535159,
       2823.47073292, 2822.65570919, 2821.15170238, 2820.26291547,
       2820.74044902, 2821.84599195, 2822.62395538, 2821.84438924,
       2822.16837284, 2823.63097155, 2823.97617814, 2823.84736

In [ ]:
models = pd.DataFrame({
    'Modelo': ['Naive Bayes', 
              'Regressão Logística',
              'Support Vector Machines',
              'Decision Tree', 
              'Random Forest', 
              'Gradient Boosting', 
              'k-Nearest Neighbors', 
              'Perceptron', 
              'Rede Neural (MLP)',
              'STACKING',
              'BLENDING'              
              ],
    
     'Gini_Treino': [gini_gaussian_train,
                    gini_logreg_train,
                    gini_svc_train,
                    gini_dectree_train,
                    gini_rndforest_train,
                    gini_gbc_train,
                    gini_knn_train,
                    gini_perceptron_train,
                    gini_mlp_train,
                    gini_stk_train,
                    gini_blend_train],   
    
    
    'Gini_Teste': [gini_gaussian_test,
                    gini_logreg_test,
                    gini_svc_test,
                    gini_dectree_test,
                    gini_rndforest_test,
                    gini_gbc_test,
                    gini_knn_test,
                    gini_perceptron_test,
                    gini_mlp_test,
                    gini_stk_test,
                    gini_blend_test]

})
models['Delta'] = models['Gini_Treino'] - models['Gini_Teste']
model_comp = models.sort_values(by='Gini_Teste', ascending=False)
model_comp = model_comp[['Modelo','Gini_Treino','Gini_Teste','Delta']]


model_comp